# 📄 Document type classification baseline code with WandB Integration



In [2]:

# =============================================================================
# 0. Prepare Environments & Install Libraries
# =============================================================================

# 필요한 라이브러리를 설치합니다.
!pip install -r ../requirements.txt

In [4]:
# =============================================================================
# 1. Import Libraries & Define Functions
# =============================================================================

import os
import time
import random
import copy

import optuna, math
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import autocast, GradScaler  # Mixed Precision용

from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# WandB 관련 import 추가
import wandb
from datetime import datetime


In [5]:
# =============================================================================
# 1-1. WandB Login and Configuration
# =============================================================================
"""
🚀 팀원 사용 가이드:

1. WandB 계정 생성: https://wandb.ai/signup
2. 이 셀 실행 시 로그인 프롬프트가 나타나면 개인 API 키 입력
3. EXPERIMENT_NAME을 다음과 같이 변경:
   - "member1-baseline"
   - "member2-augmentation-test"  
   - "member3-hyperparameter-tuning"
   등등 각자 다른 이름 사용

4. 팀 대시보드 URL: [여기에 당신의 프로젝트 URL 추가]

⚠️ 주의사항:
- 절대 API 키를 코드에 하드코딩하지 마세요
- EXPERIMENT_NAME만 변경하고 PROJECT_NAME은 그대로 두세요
- 각자 개인 계정으로 로그인해서 실험을 추가하세요
"""

# WandB 로그인 (각자 실행)
try:
    if wandb.api.api_key is None:
        print("WandB에 로그인이 필요합니다.")
        wandb.login()
    else:
        print(f"WandB 로그인 상태: {wandb.api.viewer()['username']}")
except:
    print("WandB 로그인을 진행합니다...")
    wandb.login()

# 프로젝트 설정 (각자 수정할 부분)
PROJECT_NAME = "document-classification-team"  # 모든 팀원 동일
ENTITY = None  # 각자 개인 계정 사용
EXPERIMENT_NAME = "efficientnet-b3-baseline"  # 팀원별로 변경 (예: "member1-hyperopt", "member2-augmentation")

print(f"프로젝트: {PROJECT_NAME}")
print(f"실험명: {EXPERIMENT_NAME}")
print("팀원들은 EXPERIMENT_NAME을 각자 다르게 변경해주세요!")

WandB 로그인 상태: kimsunmin0227
프로젝트: document-classification-team
실험명: efficientnet-b3-baseline
팀원들은 EXPERIMENT_NAME을 각자 다르게 변경해주세요!


In [6]:
# =============================================================================
# 3. Seed & basic augmentations (Mixup)
# =============================================================================

# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

# Mixup 함수 정의
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).cuda()
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# 추가적인 augmentations 있는가 ??? 


In [14]:

# =============================================================================
# 4. Dataset Class
# =============================================================================

class ImageDataset(Dataset):
    def __init__(self, data, path, transform=None):
        # CSV 파일이면 읽고, DataFrame이면 그대로 사용
        if isinstance(data, str):
            self.df = pd.read_csv(data).values
        else:
            self.df = data.values  # DataFrame을 numpy array로 변환
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target


In [15]:

# =============================================================================
# 5. Training Functions with WandB Logging
# =============================================================================

def train_one_epoch(loader, model, optimizer, loss_fn, device, epoch=None, fold=None):
    scaler = GradScaler()
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader, desc=f"Training Epoch {epoch+1 if epoch else '?'}")
    batch_count = 0
    
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)
        
        # Cutmix/Mixup 적용 (30% 확률)
        mixup_applied = False
        if random.random() < 0.3:
            mixed_x, y_a, y_b, lam = mixup_data(image, targets, alpha=1.0)
            with autocast(): 
                preds = model(mixed_x)
            loss = lam * loss_fn(preds, y_a) + (1 - lam) * loss_fn(preds, y_b)
            mixup_applied = True
        else:
            with autocast(): 
                preds = model(image)
            loss = loss_fn(preds, targets)

        model.zero_grad(set_to_none=True)
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        # 배치별 상세 로깅 (100 배치마다)
        if batch_count % 100 == 0 and wandb.run is not None:
            step = epoch * len(loader) + batch_count if epoch is not None else batch_count
            wandb.log({
                f"fold_{fold}/train_batch_loss": loss.item(),
                f"fold_{fold}/mixup_applied": int(mixup_applied),
                f"fold_{fold}/batch_step": step
            })
        
        batch_count += 1
        pbar.set_description(f"Loss: {loss.item():.4f}, Mixup: {mixup_applied}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret


def validate_one_epoch(loader, model, loss_fn, device, epoch=None, fold=None, log_confusion=False):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []
    
    with torch.no_grad():
        pbar = tqdm(loader, desc=f"Validating Epoch {epoch+1 if epoch else '?'}")
        for image, targets in pbar:
            image = image.to(device)
            targets = targets.to(device)
            
            preds = model(image)
            loss = loss_fn(preds, targets)
            
            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())
            
            pbar.set_description(f"Val Loss: {loss.item():.4f}")
    
    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')
    
    # Confusion Matrix 로깅 (마지막 epoch에만)
    if log_confusion and wandb.run is not None:
        try:
            wandb.log({
                f"fold_{fold}/confusion_matrix": wandb.plot.confusion_matrix(
                    probs=None,
                    y_true=targets_list,
                    preds=preds_list,
                    class_names=[f"Class_{i}" for i in range(17)]
                )
            })
            
            # 클래스별 F1 스코어
            class_f1_scores = f1_score(targets_list, preds_list, average=None)
            for i, class_f1 in enumerate(class_f1_scores):
                wandb.log({f"fold_{fold}/class_{i}_f1": class_f1})
                
        except Exception as e:
            print(f" Confusion matrix 로깅 실패: {e}")
    
    ret = {
        "val_loss": val_loss,
        "val_acc": val_acc,  
        "val_f1": val_f1,
    }
    
    return ret


In [16]:
# =============================================================================
# 6. Hyper-parameters with WandB Config
# =============================================================================

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f" Using device: {device}")

# data config
data_path = '../data/'

# model config
model_name = 'efficientnet_b3' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 384
LR = 5e-4
EPOCHS = 10
BATCH_SIZE = 32
num_workers = 30

# K-Fold config
N_FOLDS = 5  # 5-fold로 설정

# WandB Config 설정
config = {
    # Model config
    "model_name": model_name,
    "img_size": img_size,
    "num_classes": 17,
    "architecture": "EfficientNet-B3",
    
    # Training config  
    "lr": LR,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "num_workers": num_workers,
    "device": str(device),
    
    # K-Fold config
    "n_folds": N_FOLDS,
    "seed": SEED,
    "cv_strategy": "StratifiedKFold",
    
    # Augmentation & Training techniques
    "mixup_alpha": 1.0,
    "mixup_prob": 0.3,
    "label_smoothing": 0.2,
    "gradient_clipping": 1.0,
    "mixed_precision": True,
    
    # Optimizer & Scheduler
    "optimizer": "Adam",
    "scheduler": "CosineAnnealingLR",
    
    # Data
    "data_path": data_path,
    "train_transforms": "Advanced",
    "test_transforms": "Basic",
}

print(" 하이퍼파라미터 설정 완료!")
print(f" 모델: {model_name}")
print(f" 이미지 크기: {img_size}x{img_size}")
print(f" 배치 크기: {BATCH_SIZE}")
print(f" 학습률: {LR}")
print(f" 에폭: {EPOCHS}")


 Using device: cuda
 하이퍼파라미터 설정 완료!
 모델: efficientnet_b3
 이미지 크기: 384x384
 배치 크기: 32
 학습률: 0.0005
 에폭: 10


In [17]:

# =============================================================================
# 7. Optuna Hyperparameter Tuning (선택적)
# =============================================================================

USE_OPTUNA = False  # True로 바꾸면 튜닝 실행

if USE_OPTUNA:
    print("🔍 Optuna 하이퍼파라미터 튜닝 시작...")
    
    def objective(trial):
        lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
        
        # WandB에 Optuna 시행 로깅
        optuna_run = wandb.init(
            project=PROJECT_NAME,
            entity=ENTITY,
            name=f"optuna-trial-{trial.number}",
            config={**config, "lr": lr, "batch_size": batch_size},
            tags=["optuna", "hyperparameter-tuning"],
            group="optuna-study",
            job_type="hyperparameter-optimization",
            reinit=True
        )
        
        # 간단한 3-fold CV로 빠른 평가
        skf_simple = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        fold_scores = []
        
        # 간단한 평가 로직 (실제 구현에서는 더 단순화)
        # ... (Optuna 로직은 복잡하므로 기본적으로 비활성화)
        
        optuna_run.finish()
        return np.random.random()  # placeholder
    
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=10)
    
    # 최적 파라미터 적용
    best_params = study.best_params
    LR = best_params.get('lr', LR)
    BATCH_SIZE = best_params.get('batch_size', BATCH_SIZE)
    config.update(best_params)
    print(f"🎯 Optuna 최적 파라미터: {best_params}")
else:
    print("⏭️ Optuna 튜닝 건너뛰기 (USE_OPTUNA=False)")

⏭️ Optuna 튜닝 건너뛰기 (USE_OPTUNA=False)


In [18]:
# =============================================================================
# 8. Data Transforms
# =============================================================================

# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 비율 보존 리사이징 (핵심 개선)
    A.LongestMaxSize(max_size=img_size),
    A.PadIfNeeded(min_height=img_size, min_width=img_size, 
                  border_mode=0, value=0),
    
    # 문서 특화 회전 (정확한 90도 배수)
    A.OneOf([
        A.Rotate(limit=[90,90], p=1.0),
        A.Rotate(limit=[180,180], p=1.0),
        A.Rotate(limit=[270,270], p=1.0),
    ], p=0.6),
    
    # 테스트 특화 강화 증강
    A.OneOf([
        A.MotionBlur(blur_limit=7, p=1.0),
        A.GaussianBlur(blur_limit=7, p=1.0),
    ], p=0.9),
    
    A.RandomBrightnessContrast(
        brightness_limit=0.3, 
        contrast_limit=0.3, 
        p=0.8
    ),
    A.GaussNoise(var_limit=(30.0, 100.0), p=0.7),
    A.HorizontalFlip(p=0.5),
    
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.LongestMaxSize(max_size=img_size),
    A.PadIfNeeded(min_height=img_size, min_width=img_size, 
                  border_mode=0, value=0),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

print("✅ 데이터 변환 설정 완료!")

✅ 데이터 변환 설정 완료!


In [19]:
# =============================================================================
# 9. Load Data & Start K-Fold Cross Validation with WandB
# =============================================================================

# 전체 학습 데이터 로드
train_df = pd.read_csv("../data/train.csv")
print(f"📊 학습 데이터: {len(train_df)}개 샘플")

# 클래스 분포 확인
class_counts = train_df['target'].value_counts().sort_index()
print(f"📊 클래스 분포: {dict(class_counts)}")

# K-Fold 설정
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

# K-Fold 결과를 저장할 리스트
fold_results = []
fold_models = []  # 각 fold의 최고 성능 모델을 저장

# 🔥 WandB 메인 실험 시작
main_run = wandb.init(
    project=PROJECT_NAME,
    entity=ENTITY,
    name=f"{EXPERIMENT_NAME}-{datetime.now().strftime('%m%d-%H%M')}",
    config=config,
    tags=["k-fold-cv", "ensemble", model_name, "baseline", "main-experiment"],
    group="k-fold-experiment",
    job_type="cross-validation",
    notes=f"{N_FOLDS}-Fold Cross Validation with {model_name}"
)

print(f"\n🚀 WandB 실험 시작!")
print(f"📊 대시보드: {main_run.url}")
print(f"📋 실험명: {main_run.name}")

# 🔥 데이터셋 정보 로깅
wandb.log({
    "dataset/total_samples": len(train_df),
    "dataset/num_classes": 17,
    "dataset/samples_per_fold": len(train_df) // N_FOLDS,
})

# 클래스 분포 시각화
class_dist_data = [[f"Class_{i}", count] for i, count in enumerate(class_counts)]
wandb.log({
    "dataset/class_distribution": wandb.plot.bar(
        wandb.Table(data=class_dist_data, columns=["Class", "Count"]),
        "Class", "Count", 
        title="Training Data Class Distribution"
    )
})

print(f"\n{'='*60}")
print(f"🎯 {N_FOLDS}-FOLD CROSS VALIDATION 시작")
print(f"{'='*60}")


📊 학습 데이터: 1570개 샘플
📊 클래스 분포: {0: 100, 1: 46, 2: 100, 3: 100, 4: 100, 5: 100, 6: 100, 7: 100, 8: 100, 9: 100, 10: 100, 11: 100, 12: 100, 13: 74, 14: 50, 15: 100, 16: 100}


wandb: Currently logged in as: kimsunmin0227 (kimsunmin0227-hufs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



🚀 WandB 실험 시작!
📊 대시보드: https://wandb.ai/kimsunmin0227-hufs/document-classification-team/runs/3jonazfk
📋 실험명: efficientnet-b3-baseline-0904-0322

🎯 5-FOLD CROSS VALIDATION 시작


In [20]:

# =============================================================================
# 10. K-Fold Cross Validation Loop with WandB
# =============================================================================

for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['target'])):
    print(f"\n{'='*50}")
    print(f" FOLD {fold + 1}/{N_FOLDS}")
    print(f"{'='*50}")
    
    # 각 fold별 child run 생성
    fold_run = wandb.init(
        project=PROJECT_NAME,
        entity=ENTITY,
        name=f"fold-{fold+1}-{model_name}-{datetime.now().strftime('%H%M')}",
        config=config,
        tags=["fold", f"fold-{fold+1}", model_name, "child-run"],
        group="k-fold-experiment",
        job_type=f"fold-{fold+1}",
        reinit=True  # 새로운 run 시작 허용
    )
    
    print(f"📊 Fold {fold+1} Dashboard: {fold_run.url}")
    
    # 현재 fold의 train/validation 데이터 분할
    train_fold_df = train_df.iloc[train_idx].reset_index(drop=True)
    val_fold_df = train_df.iloc[val_idx].reset_index(drop=True)
    
    # 데이터 분할 정보 로깅
    wandb.log({
        "fold_info/fold_number": fold + 1,
        "fold_info/train_samples": len(train_fold_df),
        "fold_info/val_samples": len(val_fold_df),
        "fold_info/train_ratio": len(train_fold_df) / len(train_df),
        "fold_info/val_ratio": len(val_fold_df) / len(train_df)
    })
    
    # 현재 fold의 Dataset 생성
    trn_dataset = ImageDataset(
        train_fold_df,
        "../data/train/",
        transform=trn_transform
    )
    
    val_dataset = ImageDataset(
        val_fold_df,
        "../data/train/",
        transform=tst_transform  # 검증에는 증강 적용 안함
    )
    
    # 현재 fold의 DataLoader 생성
    trn_loader = DataLoader(
        trn_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )
    
    print(f"Train samples: {len(trn_dataset)}, Validation samples: {len(val_dataset)}")
    
    # 모델 초기화 (각 fold마다 새로운 모델)
    model = timm.create_model(
        model_name,
        pretrained=True,
        num_classes=17
    ).to(device)
    
    loss_fn = nn.CrossEntropyLoss(label_smoothing=0.2)  # Label Smoothing 적용
    optimizer = Adam(model.parameters(), lr=LR)
    
    # Learning Rate Scheduler 추가
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)
    
    # 현재 fold의 최고 성능 추적
    best_val_f1 = 0.0
    best_model = None
    patience = 0
    max_patience = 3
    
    print(f" 모델 학습 시작 - Fold {fold+1}")
    
    # =============================================================================
    # 11. Training Loop for Current Fold
    # =============================================================================
    
    for epoch in range(EPOCHS):
        print(f"\n📈 Epoch {epoch+1}/{EPOCHS}")
        
        # Training
        train_ret = train_one_epoch(
            trn_loader, model, optimizer, loss_fn, device, 
            epoch=epoch, fold=fold+1
        )
        
        # Validation
        val_ret = validate_one_epoch(
            val_loader, model, loss_fn, device, 
            epoch=epoch, fold=fold+1,
            log_confusion=(epoch == EPOCHS-1)  # 마지막 epoch에만 confusion matrix
        )
        
        # Learning rate 로깅
        current_lr = optimizer.param_groups[0]['lr']
        
        # WandB에 metrics 로깅
        log_data = {
            "epoch": epoch + 1,
            "fold": fold + 1,
            "train/loss": train_ret['train_loss'],
            "train/accuracy": train_ret['train_acc'], 
            "train/f1": train_ret['train_f1'],
            "val/loss": val_ret['val_loss'],
            "val/accuracy": val_ret['val_acc'],
            "val/f1": val_ret['val_f1'],
            "learning_rate": current_lr,
            "optimizer/lr": current_lr
        }
        
        # GPU 메모리 사용량 로깅
        if torch.cuda.is_available():
            gpu_memory_used = torch.cuda.memory_allocated(0) / 1e9
            gpu_memory_total = torch.cuda.get_device_properties(0).total_memory / 1e9
            log_data.update({
                "system/gpu_memory_used_gb": gpu_memory_used,
                "system/gpu_memory_total_gb": gpu_memory_total,
                "system/gpu_utilization_pct": (gpu_memory_used / gpu_memory_total) * 100
            })
        
        wandb.log(log_data)
        
        # Scheduler step
        scheduler.step()
        
        print(f" Epoch {epoch+1:2d} | "
              f"Train Loss: {train_ret['train_loss']:.4f} | "
              f"Train F1: {train_ret['train_f1']:.4f} | "
              f"Val Loss: {val_ret['val_loss']:.4f} | "
              f"Val F1: {val_ret['val_f1']:.4f} | "
              f"LR: {current_lr:.2e}")
        
        # 최고 성능 모델 저장
        if val_ret['val_f1'] > best_val_f1:
            best_val_f1 = val_ret['val_f1']
            best_model = copy.deepcopy(model.state_dict())
            patience = 0
            
            # 최고 성능 모델 아티팩트로 저장
            model_path = f'best_model_fold_{fold+1}.pth'
            torch.save(best_model, model_path)
            wandb.save(model_path, policy="now")
            
            # 새로운 최고 성능 로깅
            wandb.log({
                f"best_performance/epoch": epoch + 1,
                f"best_performance/val_f1": best_val_f1,
                f"best_performance/val_acc": val_ret['val_acc'],
                f"best_performance/val_loss": val_ret['val_loss'],
            })
            
            print(f"🎉 새로운 최고 성능! F1: {best_val_f1:.4f}")
        else:
            patience += 1
            
        # Early stopping (선택적)
        if patience >= max_patience and epoch > EPOCHS // 2:
            print(f"⏸️ Early stopping at epoch {epoch+1} (patience: {patience})")
            wandb.log({"early_stopping/epoch": epoch + 1})
            break
    
    # =============================================================================
    # 12. Fold Results Summary
    # =============================================================================
    
    # 현재 fold 결과 저장
    fold_result = {
        'fold': fold + 1,
        'best_val_f1': best_val_f1,
        'final_train_f1': train_ret['train_f1'],
        'train_samples': len(trn_dataset),
        'val_samples': len(val_dataset),
        'epochs_trained': epoch + 1,
        'early_stopped': patience >= max_patience
    }
    
    fold_results.append(fold_result)
    fold_models.append(best_model)
    
    # Fold 최종 요약 로깅
    wandb.log({
        "fold_summary/best_val_f1": best_val_f1,
        "fold_summary/final_train_f1": train_ret['train_f1'],
        "fold_summary/epochs_trained": epoch + 1,
        "fold_summary/improvement": best_val_f1 - val_ret['val_f1'],
        "fold_summary/early_stopped": patience >= max_patience
    })
    
    print(f"\n Fold {fold + 1} 완료!")
    print(f" 최고 Validation F1: {best_val_f1:.4f}")
    print(f" 학습된 에폭: {epoch + 1}/{EPOCHS}")
    
    # Fold run 종료
    wandb.finish()
    
    # 메모리 정리
    del model, optimizer, scheduler, trn_loader, val_loader
    torch.cuda.empty_cache()



 FOLD 1/5


dataset/num_classes,▁
dataset/samples_per_fold,▁
dataset/total_samples,▁
dataset/num_classes,17
dataset/samples_per_fold,314
dataset/total_samples,1570


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


📊 Fold 1 Dashboard: https://wandb.ai/kimsunmin0227-hufs/document-classification-team/runs/8yiat5j1
Train samples: 1256, Validation samples: 314
 모델 학습 시작 - Fold 1

📈 Epoch 1/10


Val Loss: 1.7581: 100%|██████████| 10/10 [00:02<00:00,  4.54it/s]


...

Val Loss: 1.1097: 100%|██████████| 10/10 [00:01<00:00,  6.25it/s]


 Epoch  9 | Train Loss: 1.2494 | Train F1: 0.8502 | Val Loss: 1.2054 | Val F1: 0.8965 | LR: 4.77e-05
🎉 새로운 최고 성능! F1: 0.8965

📈 Epoch 10/10


Val Loss: 1.1207: 100%|██████████| 10/10 [00:01<00:00,  5.95it/s]


 Epoch 10 | Train Loss: 1.3393 | Train F1: 0.7544 | Val Loss: 1.2042 | Val F1: 0.8861 | LR: 1.22e-05

 Fold 5 완료!
 최고 Validation F1: 0.8965
 학습된 에폭: 10/10


best_performance/epoch,▁▂▃▅▆▇█
best_performance/val_acc,▁▃▆▇███
best_performance/val_f1,▁▃▆████
best_performance/val_loss,█▄▃▂▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
fold,▁▁▁▁▁▁▁▁▁▁
fold_5/batch_step,▁▂▃▃▄▅▆▆▇█
fold_5/class_0_f1,▁
fold_5/class_10_f1,▁
fold_5/class_11_f1,▁
+36,...


In [23]:
# =============================================================================
# 13. K-Fold Cross Validation Results Summary
# =============================================================================

print(f"\n{'='*60}")
print(" K-FOLD CROSS VALIDATION 최종 결과")
print(f"{'='*60}")

val_f1_scores = [result['best_val_f1'] for result in fold_results]
mean_f1 = np.mean(val_f1_scores)
std_f1 = np.std(val_f1_scores)

try:
    # wandb.run이 현재 활성화된 run을 가리킴
    if wandb.run is None:
        print(" 활성화된 run이 없어 새로운 summary run을 생성합니다.")
        active_run = wandb.init(
            project=PROJECT_NAME,
            name=f"SUMMARY-{EXPERIMENT_NAME}-{datetime.now().strftime('%m%d-%H%M')}",
            config=config,
            tags=["summary", "cv-results", model_name],
            group="k-fold-experiment",
            job_type="summary",
            reinit=True
        )
    else:
        print(" 기존 run을 사용합니다.")
        active_run = wandb.run
        
except Exception as e:
    print(f" Run 상태 확인 중 에러: {e}")
    # 새로운 run 생성
    active_run = wandb.init(
        project=PROJECT_NAME,
        name=f"SUMMARY-{EXPERIMENT_NAME}-{datetime.now().strftime('%m%d-%H%M')}",
        config=config,
        tags=["summary", "cv-results", model_name],
        group="k-fold-experiment",
        job_type="summary",
        reinit=True
    )

# CV 요약 테이블 생성
fold_table = wandb.Table(columns=[
    "Fold", "Best_Val_F1", "Final_Train_F1", "Train_Samples", 
    "Val_Samples", "Epochs_Trained", "Early_Stopped"
])

for result in fold_results:
    fold_table.add_data(
        result['fold'], 
        result['best_val_f1'], 
        result['final_train_f1'],
        result['train_samples'], 
        result['val_samples'],
        result['epochs_trained'],
        result['early_stopped']
    )

# 안전한 로깅
try:
    active_run.log({
        "cv_results/mean_f1": mean_f1,
        "cv_results/std_f1": std_f1,
        "cv_results/best_fold_f1": max(val_f1_scores),
        "cv_results/worst_fold_f1": min(val_f1_scores),
        "cv_results/f1_range": max(val_f1_scores) - min(val_f1_scores),
        "cv_results/fold_results_table": fold_table,
        "cv_results/n_folds": N_FOLDS,
        "cv_results/total_epochs": sum([r['epochs_trained'] for r in fold_results]),
        "cv_results/avg_epochs_per_fold": np.mean([r['epochs_trained'] for r in fold_results]),
        "cv_results/early_stopped_folds": sum([r['early_stopped'] for r in fold_results])
    })
    
    # Fold별 성능 바차트 생성
    fold_performance_data = [[f"Fold {i+1}", score] for i, score in enumerate(val_f1_scores)]
    active_run.log({
        "cv_results/fold_performance_chart": wandb.plot.bar(
            wandb.Table(data=fold_performance_data, columns=["Fold", "F1_Score"]),
            "Fold", "F1_Score", 
            title="K-Fold Cross Validation Performance"
        )
    })
    
    print(" CV 결과 로깅 완료!")
    
except Exception as e:
    print(f" WandB 로깅 중 에러: {e}")
    print(" 결과를 콘솔에 출력합니다:")

# 어떤 경우든 콘솔에는 결과 출력
for result in fold_results:
    status = " Early Stopped" if result['early_stopped'] else " Completed"
    print(f"Fold {result['fold']}: {result['best_val_f1']:.4f} "
          f"({result['epochs_trained']} epochs) {status}")

print(f"\n 평균 CV F1: {mean_f1:.4f} ± {std_f1:.4f}")
print(f" 최고 Fold: {max(val_f1_scores):.4f}")
print(f" 최악 Fold: {min(val_f1_scores):.4f}")
print(f" 성능 범위: {max(val_f1_scores) - min(val_f1_scores):.4f}")



 K-FOLD CROSS VALIDATION 최종 결과
 기존 run을 사용합니다.
 CV 결과 로깅 완료!
Fold 1: 0.9159 (8 epochs)  Early Stopped
Fold 2: 0.9431 (10 epochs)  Completed
Fold 3: 0.9150 (10 epochs)  Completed
Fold 4: 0.9104 (9 epochs)  Early Stopped
Fold 5: 0.8965 (10 epochs)  Completed

 평균 CV F1: 0.9162 ± 0.0151
 최고 Fold: 0.9431
 최악 Fold: 0.8965
 성능 범위: 0.0465


In [24]:

# =============================================================================
# 14. Ensemble Models Preparation
# =============================================================================

# 5-Fold 앙상블 모델 준비
ensemble_models = []
print(f"\n🔧 앙상블 모델 준비 중...")

for i, state_dict in enumerate(fold_models):
    fold_model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
    fold_model.load_state_dict(state_dict)
    fold_model.eval()
    ensemble_models.append(fold_model)
    print(f"Fold {i+1} 모델 로드 완료")

print(f" 총 {len(ensemble_models)}개 모델로 앙상블 구성")

try:
    if wandb.run is not None:
        wandb.run.log({
            "ensemble/num_models": len(ensemble_models),
            "ensemble/model_architecture": model_name,
            "ensemble/ensemble_type": "simple_average"
        })
    else:
        print("📊 앙상블 정보:")
        print(f"  - 모델 개수: {len(ensemble_models)}")
        print(f"  - 아키텍처: {model_name}")
        print(f"  - 앙상블 타입: simple_average")
except Exception as e:
    print(f"⚠️ 앙상블 정보 로깅 실패: {e}")



🔧 앙상블 모델 준비 중...
Fold 1 모델 로드 완료
Fold 2 모델 로드 완료
Fold 3 모델 로드 완료
Fold 4 모델 로드 완료
Fold 5 모델 로드 완료
 총 5개 모델로 앙상블 구성


In [25]:

# =============================================================================
# 15. TTA (Test Time Augmentation) Setup
# =============================================================================

# Temperature Scaling 클래스 정의
class TemperatureScaling(nn.Module):
    def __init__(self, temperature=1.5):
        super().__init__()
        self.temperature = nn.Parameter(torch.ones(1) * temperature)
    
    def forward(self, logits):
        return logits / self.temperature

print(f"\n TTA (Test Time Augmentation) 설정...")

# Essential TTA transforms
essential_tta_transforms = [
    # 원본
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    # 90도 회전들
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.Rotate(limit=[90, 90], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.Rotate(limit=[180, 180], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.Rotate(limit=[-90, -90], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    # 밝기 개선
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.RandomBrightnessContrast(brightness_limit=[0.3, 0.3], contrast_limit=[0.3, 0.3], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
]

print(f"TTA 변환 {len(essential_tta_transforms)}개 준비 완료")

try:
    if wandb.run is not None:
        wandb.run.log({
            "tta/num_transforms": len(essential_tta_transforms),
            "tta/transforms_used": ["original", "rot_90", "rot_180", "rot_270", "brightness"],
            "tta/batch_size": 64  # TTA용 배치 크기
        })
    else:
        print("📊 TTA 설정 정보:")
        print(f"  - 변형 개수: {len(essential_tta_transforms)}")
        print(f"  - 변형 종류: original, rot_90, rot_180, rot_270, brightness")
        print(f"  - 배치 크기: 64")
except Exception as e:
    print(f"⚠️ TTA 설정 로깅 실패: {e}")
    print("📊 TTA 설정 정보:")
    print(f"  - 변형 개수: {len(essential_tta_transforms)}")
    print(f"  - 배치 크기: 64")


 TTA (Test Time Augmentation) 설정...
TTA 변환 5개 준비 완료


In [26]:
# =============================================================================
# 16. TTA Dataset and DataLoader
# =============================================================================

class TTAImageDataset(Dataset):
    def __init__(self, data, path, transforms):
        if isinstance(data, str):
            self.df = pd.read_csv(data).values
        else:
            self.df = data.values
        self.path = path
        self.transforms = transforms  # 여러 transform을 리스트로 받음

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        
        # 모든 transform을 적용한 결과를 리스트로 반환
        augmented_images = []
        for transform in self.transforms:
            aug_img = transform(image=img)['image']
            augmented_images.append(aug_img)
        
        return augmented_images, target

# TTA Dataset 생성
tta_dataset = TTAImageDataset(
    "../data/sample_submission.csv",
    "../data/test/",
    essential_tta_transforms
)

# TTA DataLoader (배치 크기를 줄여서 메모리 절약)
tta_loader = DataLoader(
    tta_dataset,
    batch_size=64,  # TTA는 메모리를 많이 사용하므로 배치 크기 줄임
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

print(f" TTA Dataset: {len(tta_dataset)}개 테스트 샘플")

 TTA Dataset: 3140개 테스트 샘플


In [27]:

# =============================================================================
# 17. Ensemble + TTA Inference with WandB Logging
# =============================================================================

def ensemble_tta_inference_with_logging(models, loader, transforms, confidence_threshold=0.9):
    """5-Fold 모델 앙상블 + TTA 추론 with WandB 로깅"""
    all_predictions = []
    all_confidences = []
    
    # TTA 진행상황 로깅을 위한 테이블
    tta_progress = wandb.Table(columns=["Batch", "Avg_Confidence", "Low_Conf_Count", "High_Conf_Count"])
    
    # Temperature scaling 초기화
    temp_scaling = TemperatureScaling().to(device)
    
    print(f"앙상블 TTA 추론 시작...")
    print(f"{len(models)}개 모델 × {len(transforms)}개 TTA 변형 = {len(models) * len(transforms)}개 예측 평균")
    
    start_time = time.time()
    
    for batch_idx, (images_list, _) in enumerate(tqdm(loader, desc="Ensemble TTA")):
        batch_size = images_list[0].size(0)
        ensemble_probs = torch.zeros(batch_size, 17).to(device)
        
        # 각 fold 모델별 예측
        for model_idx, model in enumerate(models):
            model.eval()
            with torch.no_grad():
                # 각 TTA 변형별 예측
                for tta_idx, images in enumerate(images_list):
                    images = images.to(device)
                    preds = model(images)
                    
                    # Temperature scaling 적용
                    preds = temp_scaling(preds)
                    probs = torch.softmax(preds, dim=1)
                    
                    # 앙상블 확률에 누적 (평균)
                    ensemble_probs += probs / (len(models) * len(images_list))
        
        # 신뢰도 계산
        max_probs = torch.max(ensemble_probs, dim=1)[0]
        batch_confidences = max_probs.cpu().numpy()
        all_confidences.extend(batch_confidences)
        
        final_preds = torch.argmax(ensemble_probs, dim=1)
        all_predictions.extend(final_preds.cpu().numpy())
        
        # 배치별 신뢰도 분석
        high_conf_count = np.sum(batch_confidences >= confidence_threshold)
        low_conf_count = batch_size - high_conf_count
        avg_confidence = np.mean(batch_confidences)
        
        # 진행상황 테이블에 추가
        tta_progress.add_data(batch_idx, avg_confidence, low_conf_count, high_conf_count)
        
        # 배치별 상세 로깅 (20배치마다)
        if batch_idx % 20 == 0:
            elapsed_time = time.time() - start_time
            estimated_total = elapsed_time * len(loader) / (batch_idx + 1)
            remaining_time = estimated_total - elapsed_time
            
            wandb.log({
                "tta_progress/batch": batch_idx,
                "tta_progress/avg_confidence": avg_confidence,
                "tta_progress/high_confidence_ratio": high_conf_count / batch_size,
                "tta_progress/low_confidence_count": low_conf_count,
                "tta_progress/elapsed_time_min": elapsed_time / 60,
                "tta_progress/estimated_remaining_min": remaining_time / 60,
                "tta_progress/samples_processed": (batch_idx + 1) * batch_size,
            })
    
    total_time = time.time() - start_time
    
    # TTA 최종 결과 로깅
    final_avg_confidence = np.mean(all_confidences)
    confidence_std = np.std(all_confidences)
    high_conf_samples = np.sum(np.array(all_confidences) >= confidence_threshold)
    
    wandb.log({
        "tta_results/total_time_min": total_time / 60,
        "tta_results/samples_per_second": len(all_predictions) / total_time,
        "tta_results/final_avg_confidence": final_avg_confidence,
        "tta_results/confidence_std": confidence_std,
        "tta_results/high_confidence_samples": high_conf_samples,
        "tta_results/high_confidence_ratio": high_conf_samples / len(all_predictions),
        "tta_results/total_predictions": len(all_predictions),
        "tta_results/confidence_histogram": wandb.Histogram(all_confidences),
        "tta_results/progress_table": tta_progress
    })
    
    print(f"\n 앙상블 TTA 추론 완료!")
    print(f"총 소요시간: {total_time/60:.1f}분")
    print(f" 평균 신뢰도: {final_avg_confidence:.4f} ± {confidence_std:.4f}")
    print(f" 고신뢰도 샘플: {high_conf_samples}/{len(all_predictions)} ({high_conf_samples/len(all_predictions)*100:.1f}%)")
    
    return all_predictions, all_confidences

# 앙상블 TTA 실행
print(f"\n{'='*60}")
print(" 최종 추론 - 앙상블 + TTA")
print(f"{'='*60}")

tta_predictions, confidences = ensemble_tta_inference_with_logging(
    models=ensemble_models, 
    loader=tta_loader, 
    transforms=essential_tta_transforms,
    confidence_threshold=0.9
)



 최종 추론 - 앙상블 + TTA
앙상블 TTA 추론 시작...
5개 모델 × 5개 TTA 변형 = 25개 예측 평균


Ensemble TTA:   0%|          | 0/50 [00:00<?, ?it/s]

: 

In [ ]:
# =============================================================================
# 18. Final Results and Submission
# =============================================================================

print(f"\n 최종 결과 정리 중...")

# TTA 결과로 submission 파일 생성
tta_pred_df = pd.DataFrame(tta_dataset.df, columns=['ID', 'target'])
tta_pred_df['target'] = tta_predictions

# 기존 submission과 동일한 순서인지 확인
sample_submission_df = pd.read_csv("../data/sample_submission.csv")
assert (sample_submission_df['ID'] == tta_pred_df['ID']).all(), "ID 순서 불일치!"

# 예측 분포 분석
pred_distribution = tta_pred_df['target'].value_counts().sort_index()
pred_table = wandb.Table(columns=["Class", "Count", "Percentage"])

print(f"\n📊 예측 결과 분포:")
for class_id in range(17):
    count = pred_distribution.get(class_id, 0)
    percentage = count / len(tta_pred_df) * 100
    pred_table.add_data(class_id, count, percentage)
    print(f"Class {class_id:2d}: {count:4d} ({percentage:5.1f}%)")

# 신뢰도 분석
confidence_bins = [0.5, 0.7, 0.8, 0.9, 0.95, 1.0]
confidence_analysis = {}
for i, threshold in enumerate(confidence_bins):
    if i == 0:
        count = np.sum(np.array(confidences) >= threshold)
    else:
        prev_threshold = confidence_bins[i-1]
        count = np.sum((np.array(confidences) >= prev_threshold) & (np.array(confidences) < threshold))
    confidence_analysis[f"conf_{threshold}"] = count

# 최종 결과 로깅
try:
    if wandb.run is not None:
        wandb.run.log({
            "final_results/total_predictions": len(tta_predictions),
            "final_results/unique_classes_predicted": len(np.unique(tta_predictions)),
            "final_results/prediction_distribution_table": pred_table,
            "final_results/avg_confidence": np.mean(confidences),
            "final_results/median_confidence": np.median(confidences),
            "final_results/min_confidence": np.min(confidences),
            "final_results/max_confidence": np.max(confidences),
            "final_results/confidence_distribution": wandb.Histogram(confidences),
            **confidence_analysis
        })
        print("최종 결과 WandB 로깅 완료!")
    else:
        print("활성화된 run이 없어 로깅을 건너뜁니다.")
except Exception as e:
    print(f"WandB 로깅 중 에러: {e}")

# 콘솔 출력은 항상 실행
print(f"총 예측 수: {len(tta_predictions)}")
print(f"예측된 클래스 수: {len(np.unique(tta_predictions))}")
print(f"평균 신뢰도: {np.mean(confidences):.4f}")
print(f"신뢰도 범위: {np.min(confidences):.4f} ~ {np.max(confidences):.4f}")


# 예측 분포 바차트
try:
    if wandb.run is not None:
        pred_dist_data = [[f"Class_{i}", pred_distribution.get(i, 0)] for i in range(17)]
        wandb.run.log({
            "final_results/prediction_distribution_chart": wandb.plot.bar(
                wandb.Table(data=pred_dist_data, columns=["Class", "Count"]),
                "Class", "Count", 
                title="Final Prediction Distribution"
            )
        })
        print("예측 분포 차트 로깅 완료!")
    else:
        print("차트 로깅을 건너뜁니다.")
except Exception as e:
    print(f"차트 로깅 중 에러: {e}")

# 결과 저장
output_path = "../output/choice.csv"
tta_pred_df.to_csv(output_path, index=False)

# 결과 파일을 WandB 아티팩트로 저장
artifact = wandb.Artifact(
    name="final_predictions",
    type="predictions",
    description=f"Final ensemble predictions with {N_FOLDS}-fold CV + TTA"
)
artifact.add_file(output_path)

try:
    if wandb.run is not None:
        wandb.run.log_artifact(artifact)
        print("실험 요약 로깅 완료!")
    else:
        print("활성화된 run이 없어 실험 요약 로깅을 건너뜁니다.")
except Exception as e:
    print(f"실험 요약 로깅 중 에러: {e}")


print(f"\n 최종 결과 저장 완료!")
print(f" 파일 위치: {output_path}")
print(f" 총 예측 수: {len(tta_predictions)}")

In [ ]:
# =============================================================================
# 19. Experiment Summary and Cleanup
# =============================================================================

# 실험 요약 생성
experiment_summary = {
    "experiment_name": main_run.name,
    "model_architecture": model_name,
    "image_size": img_size,
    "cv_strategy": f"{N_FOLDS}-Fold StratifiedKFold",
    "cv_mean_f1": mean_f1,
    "cv_std_f1": std_f1,
    "cv_best_fold": max(val_f1_scores),
    "ensemble_models": len(ensemble_models),
    "tta_transforms": len(essential_tta_transforms),
    "total_training_time_min": sum([r['epochs_trained'] for r in fold_results]) * 2,  # 추정치
    "avg_prediction_confidence": np.mean(confidences),
    "high_confidence_predictions": np.sum(np.array(confidences) >= 0.9),
    "experiment_tags": ["baseline", "efficientnet-b3", "k-fold-cv", "tta", "ensemble"]
}

# 실험 요약
try:
    if wandb.run is not None:
        wandb.run.log({"experiment_summary": experiment_summary})
        print("실험 요약 로깅 완료!")
    else:
        print("활성화된 run이 없어 실험 요약 로깅을 건너뜁니다.")
except Exception as e:
    print(f"실험 요약 로깅 중 에러: {e}")


# 마지막 상태 업데이트
try:
    if wandb.run is not None:
        wandb.run.log({
            "status": "completed",
            "completion_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "total_runtime_hours": 0  # start_time 속성 문제로 일단 0으로 설정
        })
        print("최종 상태 업데이트 완료!")
    else:
        print("활성화된 run이 없어 상태 업데이트를 건너뜁니다.")
except Exception as e:
    print(f"상태 업데이트 중 에러: {e}")

print(f"\n실험 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n{'='*60}")
print("실험 완료!")
print(f"{'='*60}")

print(f" K-Fold CV 결과: {mean_f1:.4f} ± {std_f1:.4f}")
print(f" 최고 성능 Fold: {max(val_f1_scores):.4f}")
print(f" 앙상블 모델: {len(ensemble_models)}개")
print(f" TTA 변형: {len(essential_tta_transforms)}개")
print(f" 평균 예측 신뢰도: {np.mean(confidences):.4f}")
print(f" WandB 대시보드: {main_run.url}")

# Sample predictions 출력
print(f"\n 예측 결과 샘플:")
print(tta_pred_df.head(10))

# 메인 run 종료
main_run.finish()

print(f"\n 모든 작업 완료!")
print(f" 결과 파일: {output_path}")
print(f" WandB에서 전체 실험 결과를 확인하세요!")

# 메모리 정리
del ensemble_models
torch.cuda.empty_cache()


 최종 결과 정리 중...

📊 예측 결과 분포:
Class  0:  209 (  6.7%)
Class  1:   88 (  2.8%)
Class  2:  199 (  6.3%)
Class  3:  130 (  4.1%)
Class  4:  254 (  8.1%)
Class  5:  194 (  6.2%)
Class  6:  289 (  9.2%)
Class  7:  190 (  6.1%)
Class  8:  202 (  6.4%)
Class  9:  200 (  6.4%)
Class 10:  223 (  7.1%)
Class 11:  159 (  5.1%)
Class 12:  186 (  5.9%)
Class 13:  184 (  5.9%)
Class 14:   34 (  1.1%)
Class 15:  199 (  6.3%)
Class 16:  200 (  6.4%)
최종 결과 WandB 로깅 완료!
총 예측 수: 3140
예측된 클래스 수: 17
평균 신뢰도: 0.3468
신뢰도 범위: 0.0947 ~ 0.6534
예측 분포 차트 로깅 완료!
실험 요약 로깅 완료!

 최종 결과 저장 완료!
 파일 위치: ../output/choice.csv
 총 예측 수: 3140
실험 요약 로깅 완료!
최종 상태 업데이트 완료!

실험 완료 시간: 2025-09-04 03:03:31

실험 완료!
 K-Fold CV 결과: 0.9015 ± 0.0027
 최고 성능 Fold: 0.9056
 앙상블 모델: 5개
 TTA 변형: 5개
 평균 예측 신뢰도: 0.3468
 WandB 대시보드: https://wandb.ai/kimsunmin0227-hufs/document-classification-team/runs/jewojmhb

 예측 결과 샘플:
                     ID  target
0  0008fdb22ddce0ce.jpg       2
1  00091bffdffd83de.jpg      12
2  00396fbc1f6cc21d.jpg       

In [1]:
# print(main_run.url)
# =============================================================================
# 20. Team Collaboration Guide (마지막으로 한번 더)
# =============================================================================
 
team_guide = f"""
 팀원들을 위한 WandB 사용 가이드:

1. WandB 계정 생성 및 로그인:
   - https://wandb.ai 에서 계정 생성
   - 터미널에서: wandb login

2. 같은 프로젝트에 실험 추가:
   - PROJECT_NAME = "{PROJECT_NAME}"
   - 각자 다른 실험명 사용 (예: "member1-experiment")
   
3. 실험 태그 규칙:
   - [멤버명, 실험타입, 모델명, 기타] 형식
   - 예: ["john", "hyperparameter-tuning", "efficientnet-b4"]

4. 이 베이스라인 활용:
   - model_name, LR, BATCH_SIZE 등 하이퍼파라미터 변경
   - config 딕셔너리에 새로운 설정 추가
   - wandb.init()의 name과 tags 수정

5. 팀 대시보드 확인:
   - {main_run.url if 'main_run' in locals() else 'WandB URL을 확인하세요'}
   
현재 베이스라인 성능:
   - CV F1 Score: {mean_f1:.4f} ± {std_f1:.4f}
   - 이 성능을 기준으로 개선 실험 진행!
"""

print(team_guide)


NameError: name 'PROJECT_NAME' is not defined

In [ ]:
# 팀 가이드를 파일로도 저장
# with open("team_guide.md", "w", encoding="utf-8") as f:
#    f.write(team_guide)